# <center> <p style="font-family: Cambria; text-align: center; font-size: 48px;"> I. DATA PRE-PROCESSING </p></center>

In [330]:
#Importing all the Necessary Libraries
import pandas as pd
import numpy as np
import glob
import os
import warnings
warnings.simplefilter("ignore", UserWarning)
%matplotlib inline

## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>1. Merging all patients Diabietes Dataset into one file. </div> 

<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning:</h1>Combined all patient files into one. <br> The code loads all CSV files, assigns a Patient_ID based on the filename, and combines them into a single consolidated dataset.<br>Mergeing allows us centralised analysis of patient data, ensure consistency across records, and simplify processing, visualisation, and reporting for the entire cohort.<br> The final merged file is saved as a CSV for future analysis.</div>
 

In [332]:
# Create the Path variable to store the local path
path = "C:/work/DataAnalysis/Python Hackathon2/DataSet/HUPA-UC Diabetes Dataset"
csv_files = glob.glob(os.path.join(path, "*.csv"))

#print(csv_files)
df_list = []

#Read the csv files and append to the list
for file in csv_files:
    df = pd.read_csv(file,sep=";")
    patient_filename = os.path.basename(file)
    patient_id = os.path.splitext(patient_filename)[0]
   # print(patient_id)
    df['Patient_ID'] = patient_id
    df_list.append(df)
     
#print(df_list)

#Concatenate into one DataFrame
df_merged = pd.concat(df_list,ignore_index=True)
df_merged.to_csv("HUPA-UC_megerd.csv",index=False)
print(df_merged)

                       time     glucose  calories  heart_rate  steps  \
0       2018-06-13T18:40:00  332.000000   6.35950   82.322835   34.0   
1       2018-06-13T18:45:00  326.000000   7.72800   83.740157    0.0   
2       2018-06-13T18:50:00  330.000000   4.74950   80.525180    0.0   
3       2018-06-13T18:55:00  324.000000   6.35950   89.129032   20.0   
4       2018-06-13T19:00:00  306.000000   5.15200   92.495652    0.0   
...                     ...         ...       ...         ...    ...   
309387  2022-05-18T11:55:00  109.333333  10.79280  104.171171    0.0   
309388  2022-05-18T12:00:00  114.000000   9.80346  103.442623    0.0   
309389  2022-05-18T12:05:00  118.666667   5.66622   95.542857    0.0   
309390  2022-05-18T12:10:00  123.333333   5.57628   91.381356    0.0   
309391  2022-05-18T12:15:00  128.000000   5.57628   99.257812    0.0   

        basal_rate  bolus_volume_delivered  carb_input Patient_ID  
0         0.091667                     0.0         0.0  HUPA0001P  

## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b> 2. Setting Clinical Ranges & Checking Outliers.</div>

<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning:</h1>Outliers are data points that differ significantly from the majority of observations.<br>They can skew results, distort averages, or affect machine learning models.
<br>Identifying outliers helps in deciding whether to investigate, correct, or remove them to improve data quality and reliability of analysis.</b></p>

In [334]:
df_copy = df_merged.copy()
clip_ranges = {
    "glucose": (40, 450),          # mg/dL
    "heart_rate": (30, 200),       # bpm
    "steps": (0, 1000),             # steps in 5 min
    "calories": (0, 1000),         # kcal in 5 min
    "basal_rate": (0, 20),         # units/hour
    "bolus_volume_delivered": (0, 25), # units per 5 min
    "carb_input": (0, 300),        # grams in 5 min
}

 

for col, (low, high) in clip_ranges.items():

    if col in df_copy.columns:
        below = (df_copy[col] < low).sum() if low is not None else 0
        above = (df_copy[col] > high).sum() if high is not None else 0
        print(f"{col}: below={below}, above={above}")

glucose: below=0, above=0
heart_rate: below=0, above=0
steps: below=0, above=0
calories: below=0, above=0
basal_rate: below=0, above=0
bolus_volume_delivered: below=4, above=0
carb_input: below=0, above=0


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>2.1. Identify rows in the dataset where values fall outside the defined acceptable ranges.</div>

<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>Extracts the entire row for context, tags it with the column that triggered the outlier, the expected valid range, and the original row index, then combines all such rows into a single dataframe so you can easily review, trace back, and analyze all outliers across patients and variables in one place.</b></p>

In [336]:
outlier_rows = []

for col, (low, high) in clip_ranges.items():
    if col in df_copy.columns:
        mask = (df_copy[col] < low) | (df_copy[col] > high)
        # Pull the *entire row* from the dataframe
        outliers = df_copy.loc[mask].copy()
        
        # Add info about which column triggered the outlier
        outliers["outlier_column"] = col
        outliers["expected_range"] = f"{low}-{high}"
        outliers["row_index"] = outliers.index
        
        outlier_rows.append(outliers)

# Combine all into one dataframe
all_outliers = pd.concat(outlier_rows, ignore_index=True)

display(all_outliers)

,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,Patient_ID,outlier_column,expected_range,row_index
0,2019-03-29T15:00:00,103.0,6.48924,84.178571,0.0,0.059,-1.0,7.0,HUPA0017P,bolus_volume_delivered,0-25,46643
1,2019-03-31T14:30:00,84.0,4.77150,98.220272,0.0,0.059,-1.0,4.0,HUPA0017P,bolus_volume_delivered,0-25,47213
2,2019-04-06T12:45:00,134.0,14.60079,100.230769,84.0,0.059,-3.0,4.0,HUPA0017P,bolus_volume_delivered,0-25,48920
3,2019-04-07T13:15:00,97.0,4.77150,98.323864,0.0,0.059,-1.0,2.5,HUPA0017P,bolus_volume_delivered,0-25,49214


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>2.2. Capping Outliers with Defined Clinical Ranges</b></div>

<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>To ensures that extreme impossible values in bolus_volume_delivered metrics are adjusted rather than removed.<br>By using the clip function, all values are capped within clinically valid ranges.<br>This prevents the loss of rows that would occur if we removed data, while still keeping the dataset realistic and consistent.<br>It also provides a quick summary of how many values were corrected in each column, which is useful for data quality reporting.<br> This approach balances data integrity (no missing rows) with clinical validity (no unrealistic numbers), making the dataset more reliable for analysis and future modeling.</div>

In [338]:
for col, (low, high) in clip_ranges.items():
    if col in df_copy.columns:
        before = df_copy[col].copy()

        # Cap both lower and upper bounds
        df_copy[col] = df_copy[col].clip(lower=low, upper=high)

        # Count how many values were clipped
        clipped = (before != df_copy[col]).sum()

        print(f"{col}: capped {clipped} values outside [{low}, {high}]")


glucose: capped 0 values outside [40, 450]
heart_rate: capped 0 values outside [30, 200]
steps: capped 0 values outside [0, 1000]
calories: capped 0 values outside [0, 1000]
basal_rate: capped 0 values outside [0, 20]
bolus_volume_delivered: capped 4 values outside [0, 25]
carb_input: capped 0 values outside [0, 300]


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>3. Convert time to proper datetime format:</div>


<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>A copy of the merged dataset is created to preserve the original.<br>
The time column is converted to datetime format using pd.to_datetime().<br>
This allows us to easily extract parts of the timestamp, like the date or hour, for analysis.
</div>

In [343]:
df_copy['time'] = pd.to_datetime(df_copy['time'])
#print(df_copy)
print(df_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   time                    309392 non-null  datetime64[ns]
 1   glucose                 309392 non-null  float64       
 2   calories                309392 non-null  float64       
 3   heart_rate              309392 non-null  float64       
 4   steps                   309392 non-null  float64       
 5   basal_rate              309392 non-null  float64       
 6   bolus_volume_delivered  309392 non-null  float64       
 7   carb_input              309392 non-null  float64       
 8   Patient_ID              309392 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 21.2+ MB
None


In [301]:
df_copy

,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,Patient_ID,date
0,2018-06-13 18:40:00,332.000000,6.35950,82.322835,34.0,0.091667,0.0,0.0,HUPA0001P,2018-06-13
1,2018-06-13 18:45:00,326.000000,7.72800,83.740157,0.0,0.091667,0.0,0.0,HUPA0001P,2018-06-13
2,2018-06-13 18:50:00,330.000000,4.74950,80.525180,0.0,0.091667,0.0,0.0,HUPA0001P,2018-06-13
3,2018-06-13 18:55:00,324.000000,6.35950,89.129032,20.0,0.091667,0.0,0.0,HUPA0001P,2018-06-13
4,2018-06-13 19:00:00,306.000000,5.15200,92.495652,0.0,0.075000,0.0,0.0,HUPA0001P,2018-06-13
...,...,...,...,...,...,...,...,...,...,...
309387,2022-05-18 11:55:00,109.333333,10.79280,104.171171,0.0,0.000000,0.0,0.0,HUPA0028P,2022-05-18
309388,2022-05-18 12:00:00,114.000000,9.80346,103.442623,0.0,0.000000,0.0,0.0,HUPA0028P,2022-05-18
309389,2022-05-18 12:05:00,118.666667,5.66622,95.542857,0.0,0.000000,0.0,0.0,HUPA0028P,2022-05-18
309390,2022-05-18 12:10:00,123.333333,5.57628,91.381356,0.0,0.000000,0.0,0.0,HUPA0028P,2022-05-18


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>4. While reviewing the CGM dataset, we observed that although patients are expected to have at least 14 days of data, some patients had much longer monitoring periods, ranging from 90 days up to 2 years.</div>

In [345]:
df_copy["date"] = df_copy["time"].dt.date
patient_days = (df_copy.groupby("Patient_ID") .agg( n_days=('date', 'nunique'),
                                                     first_date=('date', 'min'),
                                                     last_date=('date', 'max') ) .reset_index()) 
patient_days
# Flag patients with >15 days
patient_days["more_than_14_days"] = patient_days["n_days"] > 14

# Sort so True comes first
df_sorted = patient_days.sort_values(by="more_than_14_days", ascending=False)

# Optional: sort within patient by date
df_sorted = df_sorted.sort_values(by=["more_than_14_days"], ascending=False)

df_sorted[df_sorted['more_than_14_days']]


,Patient_ID,n_days,first_date,last_date,more_than_14_days
0,HUPA0001P,15,2018-06-13,2018-06-27,True
23,HUPA0027P,575,2020-06-26,2022-01-21,True
22,HUPA0026P,141,2020-05-23,2020-10-10,True
24,HUPA0028P,91,2022-02-17,2022-05-18,True
21,HUPA0025P,15,2020-01-16,2020-01-30,True


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>4.1. How did we handle these cases in our analysis?</div>

<div style="color:black; font-size:18px; margin-bottom:0px;">
To ensure consistency across patients and avoid bias from those with unusually long follow-up periods, we limited the dataset to the first 15 days of data for each patient when visualizing correlations and plotting distributions.<br> This provides a fair comparison across the cohort. However, for aggregated analyses (such as computing summary statistics per patient), we retained the full monitoring period to capture all available information.</div>

## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>5. It checks whether converting the **bolus_volume_delivered** column from higher precision (**float64**) to lower precision (**float32**) changes its values.</div>


In [347]:
df_copy["bolus_volume_delivered"].astype("float64").equals(df["bolus_volume_delivered"].astype("float32"))

False

## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>5.1.This code block evaluates the **impact of converting from float64 to float32** for all numeric columns.</div>


<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1> For each column, it calculates the **maximum difference** between the original values and the converted ones. <br>- If the difference is 0, no precision is lost, making float32 safe and memory-efficient.<br> - If the difference is very small, the loss is negligible and acceptable. - If the difference is large, float64 should be kept to preserve accuracy.<br>
This ensures we balance storage optimization with data integrity before finalizing the dataset.</div>

In [350]:
float_cols = ["bolus_volume_delivered"]

for col in float_cols:
    orig = df[col].astype("float64")
    conv = df[col].astype("float32")
    max_diff = np.abs(orig - conv).max()
    print(f"{col}: max rounding difference = {max_diff}")

bolus_volume_delivered: max rounding difference = 0.0


In [352]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   time                    309392 non-null  datetime64[ns]
 1   glucose                 309392 non-null  float64       
 2   calories                309392 non-null  float64       
 3   heart_rate              309392 non-null  float64       
 4   steps                   309392 non-null  float64       
 5   basal_rate              309392 non-null  float64       
 6   bolus_volume_delivered  309392 non-null  float64       
 7   carb_input              309392 non-null  float64       
 8   Patient_ID              309392 non-null  object        
 9   date                    309392 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 23.6+ MB


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>5.2. Converting float64 to float 32</b></div>

In [354]:
df_copy = df_copy.astype({'bolus_volume_delivered':'Float32'})

In [356]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   time                    309392 non-null  datetime64[ns]
 1   glucose                 309392 non-null  float64       
 2   calories                309392 non-null  float64       
 3   heart_rate              309392 non-null  float64       
 4   steps                   309392 non-null  float64       
 5   basal_rate              309392 non-null  float64       
 6   bolus_volume_delivered  309392 non-null  Float32       
 7   carb_input              309392 non-null  float64       
 8   Patient_ID              309392 non-null  object        
 9   date                    309392 non-null  object        
dtypes: Float32(1), datetime64[ns](1), float64(6), object(2)
memory usage: 22.7+ MB


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b> 6. Conversion of Patient_ID to string type: </div>

<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>The Patient_ID column is explicitly cast to a string data type using .astype('string'). <br> This ensures consistency in data type when performing operations such as grouping, filtering, or merging.<br>Treating Patient_ID as a string preserves its role as a unique identifier for each patient’s records throughout the analysis workflow.</div>

In [360]:
df_copy['Patient_ID'] = df_copy['Patient_ID'].astype('string')
print(df_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   time                    309392 non-null  datetime64[ns]
 1   glucose                 309392 non-null  float64       
 2   calories                309392 non-null  float64       
 3   heart_rate              309392 non-null  float64       
 4   steps                   309392 non-null  float64       
 5   basal_rate              309392 non-null  float64       
 6   bolus_volume_delivered  309392 non-null  Float32       
 7   carb_input              309392 non-null  float64       
 8   Patient_ID              309392 non-null  string        
 9   date                    309392 non-null  object        
dtypes: Float32(1), datetime64[ns](1), float64(6), object(1), string(1)
memory usage: 22.7+ MB
None


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>7.Convert numeric columns to integer type</div>


<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>The columns glucose, steps, and heart_rate are explicitly converted to integers using .astype().
<br>This ensures consistent numeric data types for calculations, aggregations, and statistical analysis,
while also optimizing memory usage.</p>

In [364]:
df_copy = df_copy.astype({'glucose': int, 'steps':int,'heart_rate': int,'carb_input':int})
print(df_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   time                    309392 non-null  datetime64[ns]
 1   glucose                 309392 non-null  int32         
 2   calories                309392 non-null  float64       
 3   heart_rate              309392 non-null  int32         
 4   steps                   309392 non-null  int32         
 5   basal_rate              309392 non-null  float64       
 6   bolus_volume_delivered  309392 non-null  Float32       
 7   carb_input              309392 non-null  int32         
 8   Patient_ID              309392 non-null  string        
 9   date                    309392 non-null  object        
dtypes: Float32(1), datetime64[ns](1), float64(2), int32(4), object(1), string(1)
memory usage: 18.0+ MB
None


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>8.Round float columns to two decimal places</div>

<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>
The columns calories, basal_rate, bolus_volume_delivered, and carb_input are rounded to two decimal places using .round(). <br>
This standardizes the numeric precision, making the data easier to read and interpret, while maintaining sufficient accuracy for analysis.</div>

In [367]:
df_copy = df_copy.round({'calories': 1, 'basal_rate': 3,'bolus_volume_delivered': 1,'carb_input': 2 })
print(df_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   time                    309392 non-null  datetime64[ns]
 1   glucose                 309392 non-null  int32         
 2   calories                309392 non-null  float64       
 3   heart_rate              309392 non-null  int32         
 4   steps                   309392 non-null  int32         
 5   basal_rate              309392 non-null  float64       
 6   bolus_volume_delivered  309392 non-null  Float32       
 7   carb_input              309392 non-null  int32         
 8   Patient_ID              309392 non-null  string        
 9   date                    309392 non-null  object        
dtypes: Float32(1), datetime64[ns](1), float64(2), int32(4), object(1), string(1)
memory usage: 18.0+ MB
None


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>9.Checking for null (missing) values:</div>

<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>It is an essential step in data cleaning because missing data can lead to errors, inaccurate calculations, or biased analysis.<br>Identifying null values allows you to decide how to handle these values.</div>

In [80]:
df_copy.isnull().sum()


time                      0
glucose                   0
calories                  0
heart_rate                0
steps                     0
basal_rate                0
bolus_volume_delivered    0
carb_input                0
Patient_ID                0
dtype: int64

## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>10.Checking for the duplicate rows: </div>

<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>Duplicate rows can lead to biased results, incorrect statistics, and misleading insights.<br>Checking for duplicates ensures data integrity by identifying and removing repeated records, so the analysis reflects accurate and unique information for each observation or patient.</div>

In [372]:
# Returns True for rows that are duplicates
duplicates = df_copy.duplicated()

# Count total duplicates
print("Number of duplicate rows:", duplicates.sum())

Number of duplicate rows: 0


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>11. Rename columns to include units</div>


<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>The columns in the DataFrame are renamed to make them more readable and self-explanatory by including the measurement units.
<br>For example, glucose becomes Glucose(mg/dL), heart_rate becomes Heart Rate(BPM), and carb_input becomes Carb Intake (grams). 
<br>This helps anyone using the dataset to immediately understand the scale and units of each variable, improving clarity for analysis, visualization, or reporting.</div>

In [375]:

df_copy = df_copy.rename(columns={'time':'DateTime', 'glucose':'Glucose(mg/dL)','calories': 'Calories', 'heart_rate': 'Heart Rate(BPM)', 'steps':'Steps',
                                  'basal_rate': 'Basal Rate(units/hour)','bolus_volume_delivered':'Bolus Volume Delivered(units)',
                                  'carb_input' : 'Carb Intake (grams)'
                       })
print(df_copy)

                  DateTime  Glucose(mg/dL)  Calories  Heart Rate(BPM)  Steps  \
0      2018-06-13 18:40:00             332       6.4               82     34   
1      2018-06-13 18:45:00             326       7.7               83      0   
2      2018-06-13 18:50:00             330       4.7               80      0   
3      2018-06-13 18:55:00             324       6.4               89     20   
4      2018-06-13 19:00:00             306       5.2               92      0   
...                    ...             ...       ...              ...    ...   
309387 2022-05-18 11:55:00             109      10.8              104      0   
309388 2022-05-18 12:00:00             114       9.8              103      0   
309389 2022-05-18 12:05:00             118       5.7               95      0   
309390 2022-05-18 12:10:00             123       5.6               91      0   
309391 2022-05-18 12:15:00             128       5.6               99      0   

        Basal Rate(units/hour)  Bolus V

## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>12. Save the cleaned dataset.</div>


<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>The cleaned and processed DataFrame is exported to a CSV file named HUPA-UC_megerd_cleaned.csv using .to_csv().
<br>The 'index=False' parameter ensures that row numbers are not included in the saved file.
<br>This creates a ready-to-use dataset for further analysis or modeling.</b></p>

In [379]:
df_copy.to_csv("patients_cleaned_file.csv",index=False) 
print(df_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   DateTime                       309392 non-null  datetime64[ns]
 1   Glucose(mg/dL)                 309392 non-null  int32         
 2   Calories                       309392 non-null  float64       
 3   Heart Rate(BPM)                309392 non-null  int32         
 4   Steps                          309392 non-null  int32         
 5   Basal Rate(units/hour)         309392 non-null  float64       
 6   Bolus Volume Delivered(units)  309392 non-null  Float32       
 7   Carb Intake (grams)            309392 non-null  int32         
 8   Patient_ID                     309392 non-null  string        
 9   date                           309392 non-null  object        
dtypes: Float32(1), datetime64[ns](1), float64(2), int32(4), object(1), s

## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b> 13. Reading Demographics csv file.</div>


<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>Conversion of Patient_ID, Gender, Race to string type: 
<br>The Patient_ID, Gender, Race column is explicitly covert in to a string data type. 
<br>This ensures consistency in data type when performing operations such as grouping, filtering, or merging.
<br>Treating Patient_ID as a string preserves its role as a unique identifier for each patient’s records throughout the analysis workflow.</div>

In [382]:
df_demographics = pd.read_csv("DataSet/T1DM_patient_sleep_demographics_with_race.csv")
df_demographics = df_demographics.astype({'Patient_ID': 'string', 'Gender': 'string','Race': 'string'})
print(df_demographics.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Patient_ID                    25 non-null     string 
 1   Age                           25 non-null     int64  
 2   Gender                        25 non-null     string 
 3   Race                          25 non-null     string 
 4   Average Sleep Duration (hrs)  25 non-null     float64
 5   Sleep Quality (1-10)          25 non-null     float64
 6   % with Sleep Disturbances     25 non-null     int64  
dtypes: float64(2), int64(2), string(3)
memory usage: 1.5 KB
None


## <div style="color:darkred; font-family:'Arial',sans-seriff;font-size:26px;"><b>14. Save the demographics Dataset.</div>


<div style="color:black; font-size:18px; margin-bottom:0px;">
<h1 style="font-size:24px; font-weight:bold; margin:0 0 10px 0;">Reasoning: </h1>The cleaned and processed DataFrame is exported to a CSV file named Demographics_cleaned.csv using .to_csv().
<br>The 'index=False' parameter ensures that row numbers are not included in the saved file.
<br>This creates a ready-to-use dataset for further analysis or modeling.</b></p>

In [384]:
df_demographics.to_csv("demographics_cleaned.csv",index=False) 
print(df_demographics)

   Patient_ID  Age  Gender             Race  Average Sleep Duration (hrs)  \
0   HUPA0001P   34    Male            Other                           6.3   
1   HUPA0002P   49    Male         Hispanic                           6.6   
2   HUPA0003P   64    Male            Black                           5.3   
3   HUPA0004P   34  Female  Native American                           5.2   
4   HUPA0005P   49    Male  Native American                           5.8   
5   HUPA0006P   35    Male            White                           6.6   
6   HUPA0007P   67    Male  Native American                           7.1   
7   HUPA0009P   65  Female            Other                           6.6   
8   HUPA0010P   22    Male            Asian                           7.1   
9   HUPA0011P   63  Female            Other                           5.6   
10  HUPA0014P   56    Male            White                           6.4   
11  HUPA0015P   33  Female            Other                           6.2   